# Delta Airlines Twitter Customer Support Analysis

In this notebook we will be analyzing tweets from users for Delta Air Lines support reps using OpenAI GPT3.5 API. The analysis is done in 2 steps:

1. Intent identification: find out why a user is writing a tweet, this could be one of 4 reasons:
    - Raise a complaint/grievance
    - Ask a question
    - Share a good experience
    - Other reasons

2. Reasons for complaints: for the tweets where a complaint is being raised, we want to identify the major causes which would be one of:
    - Flight Cancellation
    - Flight Delays
    - Bad Flight Experience
    - Lost/Damaged Luggage
    - Flight Attendant Complaints
    - Flight Booking Problems
    - Poor Customer Service
    - Other

### Data Source

### Environment Setup
This notebook was run using Python 3.8 with following packages installed:
- langchain==0.0.177
- pandas==2.0.1

# Analysis

In [2]:
import pandas as pd

In [3]:
df_delta_tweets = pd.read_csv("./data/delta-airlines-tweets.csv")
df_delta_tweets.shape

(1066, 8)

## Langchain access

[API reference](https://platform.openai.com/docs/guides/chat/introduction)

In [4]:
from platform import python_version

In [9]:
# load open ai api key
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [6]:
category_prompt_template = """
you are a customer support agent for Delta Air Lines in the USA who is responding to tweets from users. given a tweet, you are supposed to classify it into one of the following categories:

- complaint
- question
- compliment

If a tweet does not fall into any of the above categories, mark it as "other".

classify the following tweet:

{tweet}
"""

In [7]:
system_message_prompt = SystemMessagePromptTemplate.from_template("")
user_message_prompt = HumanMessagePromptTemplate.from_template(category_prompt_template)
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    user_message_prompt])

In [10]:
chat = ChatOpenAI(temperature=0)

In [12]:
tweet = df_delta_tweets.text.iloc[0]
print(tweet)

@Delta I do really appreciate the app notifications when bags have been loaded on the plane. Nice to have one less thing to worry about ;)


In [13]:
ai_response = chat(chat_prompt.format_prompt(tweet=tweet).to_messages())

In [14]:
ai_response.content

'compliment'

## get all responses

In [15]:
batch_messages = [chat_prompt.format_prompt(tweet=tweet).to_messages()
                  for tweet in df_delta_tweets.text]
len(batch_messages)

1066

In [16]:
result = chat.generate(batch_messages)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 605bb6207698cd43de6d52eae46769a5 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 5110ee814aebfa66637d33cd7b1886da in your me

In [17]:
len(result.generations)

1066

### Cost

As per the [pricing](https://openai.com/pricing#language-models) documentation of Open AI, the gpt-3.5-turbo api is charged at \\$0.001 for every 1K tokens. Below we can see that we used 130611 tokens which would cost us **\\$0.26**. This is fairly low, but we only passed in a 1000 tweets. The cost would ramp up as we analyze more data.

In [18]:
result.llm_output

{'token_usage': {'prompt_tokens': 128406,
  'completion_tokens': 2205,
  'total_tokens': 130611},
 'model_name': 'gpt-3.5-turbo'}

In [19]:
df_delta_tweets.loc[:, "category_raw"] = [x[0].text for x in result.generations]

In [20]:
df_delta_tweets.category_raw.value_counts()

category_raw
complaint               425
compliment              333
question                243
other                    42
Category: Complaint       5
category: question        4
category: complaint       4
Category: Complaint.      2
Category: Question        2
Category: Question.       2
Category: question.       1
Category: question        1
Question                  1
suggestion                1
Name: count, dtype: int64

In [21]:
import re

In [22]:
def extract_category(raw):
    pattern = re.compile("compliment|complaint|question|other")
    match = pattern.search(raw.lower())
    if match is None:
        return None
    return match.group()

In [23]:
df_delta_tweets.loc[:, "category"] = df_delta_tweets.category_raw.apply(extract_category)

In [47]:
df_delta_tweets.loc[df_delta_tweets.category.isnull()]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,date,category_raw,category
928,511880,237698,True,Fri Dec 01 22:47:52 +0000 2017,"Hey @delta, ever considered offering #cancer p...",511879,NaN,2017-12-01,suggestion,None


In [24]:
df_delta_tweets.category.value_counts()

category
complaint     436
compliment    333
question      254
other          42
Name: count, dtype: int64

In [25]:
def sample_category(category, n):
    print("\n\n".join(list(df_delta_tweets.loc[df_delta_tweets.category == category].text.sample(n))))

In [29]:
sample_category("question", 5)

@Delta This is just a question from two curious passengers. Why don’t Airlines board passengers from the back of the plane to the front? (Minus first class or premium passengers). Wouldn’t that be more efficient with time and space? Just curious!

@Delta my last two flights aren’t registering my points in the app....help! 😩😅

@Delta I need help with my reservation tomorrow but can't sit on hold for the hold time of 1hr 10 minutes

hi @824 I am flying to @151272  in 19 days - can you share the link with me of flight entertainment struggling finding it.

Hi @delta what's the size of a personal item for an overseas flight? Need to carry a big boy @118117 Play 5 gift to make someone happy :) Thanks and Merry Christmas!!


In [30]:
df_delta_tweets.to_csv("./demo/delta-tweets-with-category.csv", index=False)

# exploring reasons for complaints

This is a good first start, but for an actual appication perspective we need to see where exactly we're going wrong. Let's try to classify the negative reviews further into some categories:

- Bad Flight Experience
- Flight Cancellation
- Flight Delays
- Poor Customer Service
- Damaged Luggage
- Flight Attendant Complaints
- Flight Booking Problems
- Lost Luggage
- Other

In [31]:
neg_category_prompt_template = """
you are a customer support agent for one of the largest airlines in the USA who is responding to tweets from users. 
given a tweet where a customer is complaining, you are supposed to identify the reason for the complaint and classify it into one of the following categories:

- Flight Cancellation
- Flight Delays
- Bad Flight Experience
- Lost/Damaged Luggage
- Flight Attendant Complaints
- Flight Booking Problems
- Poor Customer Service
- Other

classify the following tweet:

{tweet}
"""

In [32]:
system_message_prompt = SystemMessagePromptTemplate.from_template("")
user_message_prompt_complaint = HumanMessagePromptTemplate.from_template(neg_category_prompt_template)
chat_prompt_complaint = ChatPromptTemplate.from_messages([
    system_message_prompt,
    user_message_prompt_complaint])

In [33]:
df_complaints = df_delta_tweets.loc[df_delta_tweets.category == "complaint"]
df_complaints.shape

(436, 10)

In [34]:
batch_messages = [chat_prompt_complaint.format_prompt(tweet=tweet).to_messages()
                  for tweet in df_complaints.text]
result_complaint = chat.generate(batch_messages)
len(result_complaint.generations)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 357def4ab21ab8660ae05cba7caa78f7 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e8eac415575a6910c85396dd60004fc0 in your me

436

In [37]:
df_complaints.loc[:, "complaint_category_raw"] = [x[0].text for x in result_complaint.generations]

In [38]:
df_complaints.complaint_category_raw.value_counts()

complaint_category_raw
Category: Bad Flight Experience.                                                                                                                                                                                                                                                                         63
Category: Flight Delays                                                                                                                                                                                                                                                                                  61
Category: Flight Booking Problems                                                                                                                                                                                                                                                                        56
Category: Poor Customer Service.                                             

In [43]:
neg_cats = [
    "Flight Cancellation",
    "Flight Delays",
    "Bad Flight Experience",
    "Lost/Damaged Luggage",
    "Flight Attendant Complaints",
    "Flight Booking Problems",
    "Poor Customer Service",
    "Other"
]
def extract_complaint_category(raw):
    pattern = re.compile("|".join(neg_categories), re.IGNORECASE)
    match = pattern.search(raw.lower())
    if match is None:
        return None
    return match.group()

In [44]:
df_complaints.loc[:, "category"] = df_complaints.complaint_category_raw.apply(
    extract_complaint_category)

In [45]:
df_complaints.category.value_counts()

category
flight delays                  86
poor customer service          84
bad flight experience          84
flight booking problems        82
other                          42
lost/damaged luggage           35
flight attendant complaints    11
flight cancellation            10
Name: count, dtype: int64

In [46]:
df_complaints.loc[df_complaints.category.isnull()]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,date,category_raw,category,complaint_category_raw
269,597382,261641,True,Sun Dec 03 19:36:32 +0000 2017,@delta you board the entire flight then deplan...,597381,NaN,2017-12-03,complaint,None,Category: Flight Experience Complaints
690,490843,231927,True,Fri Dec 01 16:40:08 +0000 2017,@Delta got diverted from Asheville to Greenvil...,490842,NaN,2017-12-01,complaint,None,Category: Flight Diversion


In [48]:
df_complaints.to_csv("./demo/delta-tweets-complaint-category.csv")